In [ ]:
'''Assignment 2: Stereo disparity estimation
For the given two images, compute the disparity map. 

A disparity map is nothing but an output-image, which shows how much a pixel has moved from the one view to the other.
For example, if a pixel at location (x,y) has translated by 10 pixels, then the disparity map value at (x,y) will be 10.

Note that unlike in assignment 1, here the scene is not a 2D scene (i.e. the objects are at different depths. Thus, there will not be a global parametric motion between pixels)

To compute the disparity of each pixel, choose a window of some size (e.g. 5 x 5 to 15 x 15), around a pixel in the reference image,
Then  translate a window in the other (test) image and compute the translation value at which both the windows have the closest match.
The 'closest' match is computed based on some distance metric between the pixel value in the reference image window and the test image windows.
For some basic distance metric please see the document "stereo disparity review.pdf" uploaded above

In this particular case, the camera is only translated horizontally. 
Hence you would only have to translate the test window in the horizontal direction.'''

In [19]:
from PIL import Image
im = Image.open("scene1.row3.col3.ppm")
im.show()

In [18]:
from PIL import Image
im = Image.open("scene1.row3.col2.ppm")
im.show()

In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.ndimage.filters
def plane_sweep_ncc(im_l,im_r,start,steps,wid):
    """ Find disparity image using normalized cross-correlation. """
    m,n = im_l.shape
    # arrays to hold the different sums
    mean_l = np.zeros((m,n))
    mean_r = np.zeros((m,n))
    s = np.zeros((m,n))
    s_l = np.zeros((m,n))
    s_r = np.zeros((m,n))
    # array to hold depth planes
    dmaps = np.zeros((m,n,steps))
    # compute mean of patch
    scipy.ndimage.filters.uniform_filter(im_l,wid,mean_l)
    scipy.ndimage.filters.uniform_filter(im_r,wid,mean_r)
    # normalized images
    norm_l = im_l - mean_l
    norm_r = im_r - mean_r
    # try different disparities
    for displ in range(steps):
        # move left image to the right, compute sums
        scipy.ndimage.filters.uniform_filter(np.roll(norm_l,-displ-start)*norm_r,wid,s) # sum nominator
        scipy.ndimage.filters.uniform_filter(np.roll(norm_l,-displ-start)*np.roll(norm_l,-displ-start),wid,s_l)
        scipy.ndimage.filters.uniform_filter(norm_r*norm_r,wid,s_r) # sum denominator
        # store ncc scores
        dmaps[:,:,displ] = s/np.sqrt(s_l*s_r)
        # pick best depth for each pixel
    return np.argmax(dmaps,axis=2)


In [20]:
import stereo
import scipy
im_l = np.array(Image.open('scene1.row3.col2.ppm').convert('L'),'f')
im_r = np.array(Image.open('scene1.row3.col3.ppm').convert('L'),'f')
# starting displacement and steps
steps = 12
start = 4
# width for ncc
wid = 9
res = plane_sweep_ncc(im_l,im_r,start,steps,wid)
import scipy.misc
# scipy.misc.imsave('depth.png',res)
import imageio
from skimage import img_as_ubyte
imageio.imwrite('depth_ncc.png', res)
im = Image.open("depth_ncc.png")
im.show()

Lossy conversion from int64 to uint8. Range [0, 11]. Convert image to uint8 prior to saving to suppress this warning.


In [7]:
def plane_sweep_gauss(im_l,im_r,start,steps,wid):
    """ Find disparity image using normalized cross-correlation
    with Gaussian weighted neigborhoods. """
    m,n = im_l.shape
    # arrays to hold the different sums
    mean_l = np.zeros((m,n))
    mean_r = np.zeros((m,n))
    s = np.zeros((m,n))
    s_l = np.zeros((m,n))
    s_r = np.zeros((m,n))
    # array to hold depth planes
    dmaps = np.zeros((m,n,steps))
    # compute mean
    scipy.ndimage.filters.gaussian_filter(im_l,wid,0,mean_l)
    scipy.ndimage.filters.gaussian_filter(im_r,wid,0,mean_r)
    # normalized images
    norm_l = im_l - mean_l
    norm_r = im_r - mean_r
    # try different disparities
    for displ in range(steps):
            # move left image to the right, compute sums
            scipy.ndimage.filters.gaussian_filter(np.roll(norm_l,-displ-start)*norm_r,wid,0,s) # sum nominator
            scipy.ndimage.filters.gaussian_filter(np.roll(norm_l,-displ-start)*np.roll(norm_l,-displ-start),wid,0,s_l)
            scipy.ndimage.filters.gaussian_filter(norm_r*norm_r,wid,0,s_r) # sum denominator
            # store ncc scores
            dmaps[:,:,displ] = s/np.sqrt(s_l*s_r)
            # pick best depth for each pixel
    return np.argmax(dmaps,axis=2)

In [21]:
im_l = np.array(Image.open('scene1.row3.col2.ppm').convert('L'),'f')
im_r = np.array(Image.open('scene1.row3.col3.ppm').convert('L'),'f')
# starting displacement and steps
steps = 12
start = 4
# width for ncc
wid = 3
res = plane_sweep_gauss(im_l,im_r,start,steps,wid)
import scipy.misc
# scipy.misc.imsave('depth.png',res)
import imageio
from skimage import img_as_ubyte
imageio.imwrite('depth_gauss.png', res)
im = Image.open("depth_gauss.png")
im.show()

Lossy conversion from int64 to uint8. Range [0, 11]. Convert image to uint8 prior to saving to suppress this warning.
